In [ ]:
# Core Python libraries
import os
import json
import warnings
from datetime import datetime
import pickle

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

# Deep Learning (TensorFlow/Keras)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Image processing
from PIL import Image, ImageDraw
import cv2

# API and web requests
import requests
from urllib.parse import urlencode

# Environment configuration
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries imported successfully")

Libraries imported successfully


In [37]:
try:
    df = pd.read_csv('cyclone_databased_on_weather.csv')
    print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    
    # Handle missing values
    if df.isnull().sum().sum() > 0:
        df = df.fillna(df.mean(numeric_only=True))
    
    # Display cyclone distribution if available
    if 'cyclone_occurrence' in df.columns:
        cyclone_counts = df['cyclone_occurrence'].value_counts()
        cyclone_rate = (cyclone_counts[1] / cyclone_counts.sum() * 100)
        print(f"Cyclone rate: {cyclone_rate:.1f}%")

except FileNotFoundError:
    print("Creating sample dataset...")
    
    # Create sample weather dataset
    np.random.seed(42)
    n_samples = 1000
    
    df = pd.DataFrame({
        'Cyclone Name': [f'Cyclone_{i}' if i % 5 == 0 else 'No_Cyclone' for i in range(n_samples)],
        'Date': pd.date_range(start='2020-01-01', periods=n_samples, freq='D'),
        'Location': np.random.choice(['Bay of Bengal', 'Arabian Sea', 'Pacific Ocean', 'Atlantic Ocean'], n_samples),
        'wind_speed': np.abs(np.random.normal(15, 10, n_samples)),
        'pressure': np.abs(np.random.normal(1013, 25, n_samples)),
        'ocean_temperature': np.abs(np.random.normal(28, 5, n_samples)),
        'cyclone_occurrence': np.random.choice([0, 1], n_samples, p=[0.8, 0.2])
    })
    
    df.to_csv('cyclone_databased_on_weather.csv', index=False)
    print(f"Sample dataset created: {df.shape[0]} rows")

Dataset loaded: 200 rows, 7 columns
Cyclone rate: 50.0%


In [38]:
# Select numeric columns for features
numeric_columns = df.select_dtypes(include=[np.number]).columns

if 'cyclone_occurrence' in df.columns:
    # Features: weather parameters only (exclude target)
    feature_columns = [col for col in numeric_columns if col != 'cyclone_occurrence']
    X = df[feature_columns].copy()
    y = df['cyclone_occurrence'].copy()
    
    # Handle missing values
    if X.isnull().any().any():
        X = X.fillna(X.mean())
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Data split complete: {X_train_scaled.shape[1]} features")
    print(f"Training samples: {X_train_scaled.shape[0]}")
    print(f"Testing samples: {X_test_scaled.shape[0]}")

Data split complete: 3 features
Training samples: 160
Testing samples: 40


In [39]:
# Train Random Forest Classifier
weather_model = RandomForestClassifier(
    n_estimators=100, max_depth=10, random_state=42, n_jobs=-1
)

weather_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_test = weather_model.predict(X_test_scaled)
y_pred_proba_test = weather_model.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
roc_auc = roc_auc_score(y_test, y_pred_proba_test)

print(f"Weather Model Performance:")
print(f"  Accuracy: {test_accuracy:.3f}")
print(f"  Precision: {precision:.3f}")
print(f"  Recall: {recall:.3f}")
print(f"  F1-Score: {f1:.3f}")
print(f"  ROC AUC: {roc_auc:.3f}")

# Save model
os.makedirs('models', exist_ok=True)
with open('models/weather_model.pkl', 'wb') as f:
    pickle.dump(weather_model, f)
with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Weather model saved")

Weather Model Performance:
  Accuracy: 1.000
  Precision: 1.000
  Recall: 1.000
  F1-Score: 1.000
  ROC AUC: 1.000
Weather model saved


In [40]:
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32
EPOCHS = 20

dataset_path = "Cyclone_Dataset"
cyclone_dir = os.path.join(dataset_path, 'cyclone')
noncyclone_dir = os.path.join(dataset_path, 'noncyclone')

if os.path.exists(cyclone_dir) and os.path.exists(noncyclone_dir):
    cyclone_count = len([f for f in os.listdir(cyclone_dir) 
                        if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    noncyclone_count = len([f for f in os.listdir(noncyclone_dir) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    
    if cyclone_count > 0 and noncyclone_count > 0:
        print(f"Satellite dataset: {cyclone_count} cyclone, {noncyclone_count} non-cyclone images")
        
        # Create data generators
        train_datagen = ImageDataGenerator(
            rescale=1./255, validation_split=0.2,
            rotation_range=20, width_shift_range=0.2,
            height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2
        )
        
        train_generator = train_datagen.flow_from_directory(
            dataset_path, target_size=(IMG_HEIGHT, IMG_WIDTH),
            batch_size=BATCH_SIZE, class_mode='binary',
            subset='training', seed=42
        )
        
        validation_generator = train_datagen.flow_from_directory(
            dataset_path, target_size=(IMG_HEIGHT, IMG_WIDTH),
            batch_size=BATCH_SIZE, class_mode='binary',
            subset='validation', seed=42
        )
        
        # Create CNN model
        satellite_model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
            MaxPooling2D(2, 2),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Flatten(),
            Dropout(0.5),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(1, activation='sigmoid')
        ])
        
        satellite_model.compile(
            optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']
        )
        
        # Train model
        callbacks = [
            EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
            ModelCheckpoint('models/satellite_model_best.h5', monitor='val_accuracy', save_best_only=True)
        ]
        
        history = satellite_model.fit(
            train_generator, epochs=EPOCHS,
            validation_data=validation_generator,
            callbacks=callbacks, verbose=1
        )
        satellite_model.save('models/my_model.keras')
        satellite_training_history = history
        print("Satellite model training completed")
        
    else:
        print("Insufficient images in dataset")
        satellite_model = None
        satellite_training_history = None
else:
    print("Cyclone_Dataset directory not found")
    satellite_model = None
    satellite_training_history = None

Satellite dataset: 151 cyclone, 573 non-cyclone images
Found 580 images belonging to 2 classes.
Found 144 images belonging to 2 classes.
Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - accuracy: 0.7242 - loss: 0.7161

19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 471ms/step - accuracy: 0.7265 - loss: 0.7123 - val_accuracy: 0.7917 - val_loss: 0.5192
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 430ms/step - accuracy: 0.7996 - loss: 0.5115 - val_accuracy: 0.7917 - val_loss: 0.5132
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 410ms/step - accuracy: 0.7629 - loss: 0.5386 - val_accuracy: 0.7917 - val_loss: 0.4812
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 7s 389ms/step - accuracy: 0.7838 - loss: 0.5100 - val_accuracy: 0.7917 - val_loss: 0.4778
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 405ms/step - accuracy: 0.7816 - loss: 0.4983 - val_accuracy: 0.7917 - val_loss: 0.5193
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 7s 380ms/step - accuracy: 0.8186 - loss: 0.4882 - val_accuracy: 0.7917 - val_loss: 0.4830
Satellite model training completed


In [41]:
if satellite_model is not None and satellite_training_history is not None:
    val_loss, val_accuracy = satellite_model.evaluate(validation_generator, verbose=0)
    print(f"Satellite Model Performance:")
    print(f"  Validation Accuracy: {val_accuracy:.4f}")
    print(f"  Validation Loss: {val_loss:.4f}")
    
    # Test predictions on sample batch
    val_batch = next(validation_generator)
    val_images, val_labels = val_batch
    predictions = satellite_model.predict(val_images, verbose=0)
    predicted_classes = (predictions > 0.5).astype(int).flatten()
    batch_accuracy = accuracy_score(val_labels, predicted_classes)
    print(f"  Sample batch accuracy: {batch_accuracy:.3f}")
else:
    print("Satellite model not available for evaluation")

Satellite Model Performance:
  Validation Accuracy: 0.7917
  Validation Loss: 0.5201
  Sample batch accuracy: 0.844


In [42]:
# Load environment variables
load_dotenv('.env.local')

class CyclonePredictionConfig:
    def __init__(self):
        self.OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY', 'your_api_key_here')
        self.MAPBOX_ACCESS_TOKEN = os.getenv('MAPBOX_ACCESS_TOKEN', None)
        self.GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY', None)
        self.BING_MAPS_API_KEY = os.getenv('BING_MAPS_API_KEY', None)
        self.LOG_DIRECTORY = "logs"
        self.MODEL_DIRECTORY = "models"
        self.IMG_HEIGHT = 150
        self.IMG_WIDTH = 150
        
        os.makedirs(self.LOG_DIRECTORY, exist_ok=True)
        os.makedirs(self.MODEL_DIRECTORY, exist_ok=True)
    
    def get_risk_level(self, probability):
        if probability >= 0.8:
            return "EXTREME"
        elif probability >= 0.6:
            return "HIGH"
        elif probability >= 0.4:
            return "MODERATE"
        else:
            return "LOW"

config = CyclonePredictionConfig()

def get_weather_data_for_prediction(city_name):
    try:
        base_url = "http://api.openweathermap.org/data/2.5/weather"
        params = {'q': city_name, 'appid': config.OPENWEATHER_API_KEY, 'units': 'metric'}
        
        response = requests.get(base_url, params=params, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            features = [
                data['wind']['speed'],
                data['main']['pressure'],
                data['main']['temp']
            ]
            return np.array(features).reshape(1, -1), data
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None, None

# Load trained models
def load_trained_models():
    weather_model_loaded = None
    scaler_loaded = None
    satellite_model_loaded = None
    
    try:
        with open('models/weather_model.pkl', 'rb') as f:
            weather_model_loaded = pickle.load(f)
        print("Weather model loaded from file")
    except:
        weather_model_loaded = weather_model if 'weather_model' in globals() else None
        if weather_model_loaded:
            print("Weather model loaded from session")
    
    try:
        with open('models/scaler.pkl', 'rb') as f:
            scaler_loaded = pickle.load(f)
        print("Scaler loaded from file")
    except:
        scaler_loaded = scaler if 'scaler' in globals() else None
        if scaler_loaded:
            print("Scaler loaded from session")
    
    try:
        # Try loading with new Keras format first
        satellite_model_loaded = keras.models.load_model('models/my_model.keras')
        print("Satellite model loaded from .keras file")
    except:
        try:
            # Fallback to older .h5 format
            satellite_model_loaded = keras.models.load_model('models/satellite_model.h5')
            print("Satellite model loaded from .h5 file")
        except:
            satellite_model_loaded = satellite_model if 'satellite_model' in globals() else None
            if satellite_model_loaded:
                print("Satellite model loaded from session")
    
    return weather_model_loaded, scaler_loaded, satellite_model_loaded

weather_model_rt, scaler_rt, satellite_model_rt = load_trained_models()

print("Real-time prediction system configured")

Weather model loaded from file
Scaler loaded from file
Satellite model loaded from .keras file
Real-time prediction system configured


In [43]:
target_city = "Visakhapatnam"
weather_features, weather_raw = get_weather_data_for_prediction(target_city)

if weather_features is not None and weather_raw is not None:
    print(f"Weather data for {target_city}:")
    print(f"  Location: {weather_raw['name']}, {weather_raw['sys']['country']}")
    print(f"  Temperature: {weather_raw['main']['temp']}°C")
    print(f"  Wind Speed: {weather_raw['wind']['speed']} m/s")
    print(f"  Pressure: {weather_raw['main']['pressure']} hPa")
    print(f"  Humidity: {weather_raw['main']['humidity']}%")
    
    latitude = weather_raw['coord']['lat']
    longitude = weather_raw['coord']['lon']
    print(f"  Coordinates: {latitude:.4f}, {longitude:.4f}")
    
    # Weather prediction
    if weather_model_rt is not None and scaler_rt is not None:
        weather_features_scaled = scaler_rt.transform(weather_features)
        weather_prediction = weather_model_rt.predict_proba(weather_features_scaled)[0][1]
        weather_risk_level = config.get_risk_level(weather_prediction)
        
        print(f"Weather-based prediction:")
        print(f"  Cyclone probability: {weather_prediction:.3f}")
        print(f"  Risk level: {weather_risk_level}")
    else:
        weather_prediction = None
        weather_risk_level = "UNKNOWN"
else:
    print(f"Failed to fetch weather data for {target_city}")
    weather_prediction = None
    weather_risk_level = "UNKNOWN"
    latitude = None
    longitude = None

Weather data for Visakhapatnam:
  Location: Visakhapatnam, IN
  Temperature: 31.72°C
  Wind Speed: 5.73 m/s
  Pressure: 999 hPa
  Humidity: 58%
  Coordinates: 17.6900, 83.2093
Weather-based prediction:
  Cyclone probability: 0.520
  Risk level: MODERATE


In [44]:
class SatelliteImageryService:
    def __init__(self, config):
        self.config = config
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': 'CyclonePrediction/1.0'})
    
    def fetch_openweather_satellite(self, lat, lon, layer='clouds_new'):
        try:
            zoom = 8
            # Convert lat/lon to tile coordinates for better image quality
            tile_x = int((lon + 180.0) / 360.0 * (1 << zoom))
            tile_y = int((1.0 - np.log(np.tan(np.radians(lat)) + 1.0 / np.cos(np.radians(lat))) / np.pi) / 2.0 * (1 << zoom))
            
            url = f"http://maps.openweathermap.org/maps/2.0/weather/{layer}/{zoom}/{tile_x}/{tile_y}.png"
            params = {'appid': self.config.OPENWEATHER_API_KEY}
            
            response = self.session.get(url, params=params, timeout=15)
            
            if response.status_code == 200:
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                filename = f"{self.config.LOG_DIRECTORY}/satellite_{lat}_{lon}_{timestamp}.png"
                
                with open(filename, 'wb') as f:
                    f.write(response.content)
                return filename
            return None
        except Exception as e:
            print(f"OpenWeather satellite fetch failed: {e}")
            return None
    
    def fetch_mapbox_satellite(self, lat, lon):
        """Fetch real satellite imagery from Mapbox as backup"""
        if not self.config.MAPBOX_ACCESS_TOKEN:
            return None
        try:
            zoom = 12
            width, height = 512, 512
            url = f"https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/{lon},{lat},{zoom}/{width}x{height}@2x"
            params = {'access_token': self.config.MAPBOX_ACCESS_TOKEN}
            
            response = self.session.get(url, params=params, timeout=15)
            
            if response.status_code == 200:
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                filename = f"{self.config.LOG_DIRECTORY}/mapbox_satellite_{lat}_{lon}_{timestamp}.jpg"
                
                with open(filename, 'wb') as f:
                    f.write(response.content)
                return filename
            return None
        except Exception as e:
            print(f"Mapbox satellite fetch failed: {e}")
            return None
    
    def load_and_preprocess_image(self, image_path):
        """Load and preprocess real satellite image for CNN analysis"""
        try:
            # Load image using PIL
            img = Image.open(image_path)
            
            # Convert to RGB if necessary
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Resize to model input size
            img_resized = img.resize((self.config.IMG_WIDTH, self.config.IMG_HEIGHT))
            
            # Convert to numpy array and normalize
            img_array = np.array(img_resized) / 255.0
            
            # Add batch dimension
            img_array = np.expand_dims(img_array, axis=0)
            
            return img_array
        except Exception as e:
            print(f"Error preprocessing image: {e}")
            return None
    
    def analyze_image_for_cyclone(self, image_path=None, image_array=None):
        """Analyze real satellite image for cyclone patterns"""
        try:
            if image_path:
                # Load and preprocess real image file
                processed_image = self.load_and_preprocess_image(image_path)
            elif image_array is not None:
                # Use provided image array
                if len(image_array.shape) == 3:
                    processed_image = np.expand_dims(image_array, axis=0)
                else:
                    processed_image = image_array
            else:
                return 0.5
            
            if satellite_model_rt is not None and processed_image is not None:
                prediction = satellite_model_rt.predict(processed_image, verbose=0)
                return float(prediction[0][0])
            else:
                # Fallback: basic image analysis
                if image_path:
                    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        # Simple pattern analysis
                        cloud_density = np.mean(img) / 255.0
                        contrast = np.std(img) / 255.0
                        return min(0.9, cloud_density * 0.4 + contrast * 0.6)
                return 0.5
        except Exception as e:
            print(f"Error analyzing image: {e}")
            return 0.5

satellite_service = SatelliteImageryService(config)

if latitude is not None and longitude is not None:
    print(f"Fetching real satellite imagery for {target_city}...")
    
    # Try multiple sources for real satellite imagery
    satellite_image_path = None
    image_source = None
    
    # First try OpenWeatherMap satellite layer
    satellite_image_path = satellite_service.fetch_openweather_satellite(latitude, longitude)
    if satellite_image_path:
        image_source = "OpenWeatherMap"
        print(f"OpenWeatherMap satellite image obtained")
    else:
        # Try Mapbox satellite as backup
        satellite_image_path = satellite_service.fetch_mapbox_satellite(latitude, longitude)
        if satellite_image_path:
            image_source = "Mapbox"
            print(f"Mapbox satellite image obtained")
    
    if satellite_image_path and os.path.exists(satellite_image_path):
        print(f"Analyzing real satellite image from {image_source}...")
        
        # Analyze the real satellite image
        satellite_prediction = satellite_service.analyze_image_for_cyclone(image_path=satellite_image_path)
        
        # Display the satellite image with enhanced visualization
        try:
            img = Image.open(satellite_image_path)
            
            # Create enhanced visualization
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Original satellite image
            ax1.imshow(img)
            ax1.set_title(f'Real Satellite Image - {target_city}\nSource: {image_source}\nTimestamp: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
            ax1.axis('off')
            
            # Image analysis visualization
            img_gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
            ax2.imshow(img_gray, cmap='viridis')
            ax2.set_title(f'Cloud Pattern Analysis\nImage Size: {img.size}\nFile: {os.path.basename(satellite_image_path)}')
            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Display image properties
            print(f"Satellite Image Details:")
            print(f"  Source: {image_source}")
            print(f"  Size: {img.size[0]} x {img.size[1]} pixels")
            print(f"  Mode: {img.mode}")
            print(f"  File size: {os.path.getsize(satellite_image_path)} bytes")
            print(f"  Saved as: {satellite_image_path}")
            
        except Exception as e:
            print(f"Could not display image: {e}")
        
        if satellite_prediction is not None:
            satellite_risk_level = config.get_risk_level(satellite_prediction)
            print(f"Real satellite analysis results:")
            print(f"  Image source: {image_source}")
            print(f"  Cyclone probability: {satellite_prediction:.3f}")
            print(f"  Risk level: {satellite_risk_level}")
        else:
            satellite_risk_level = "UNKNOWN"
            print(f"Failed to analyze satellite image")
    else:
        print("No real satellite imagery available")
        print("Check API keys in .env.local file")
        satellite_prediction = None
        satellite_risk_level = "UNKNOWN"
else:
    print("No coordinates available for satellite analysis")
    satellite_prediction = None
    satellite_risk_level = "UNKNOWN"

Fetching real satellite imagery for Visakhapatnam...
No real satellite imagery available
Check API keys in .env.local file


In [45]:
# Enhanced Satellite Image Visualization and Analysis
if satellite_image_path and os.path.exists(satellite_image_path):
    print("SATELLITE IMAGE ANALYSIS DETAILS")
    print("="*50)
    
    # Load and analyze the satellite image
    img = Image.open(satellite_image_path)
    img_array = np.array(img)
    
    # Create comprehensive visualization
    fig = plt.figure(figsize=(16, 10))
    
    # Original satellite image (larger)
    ax1 = plt.subplot(2, 3, (1, 2))
    ax1.imshow(img)
    ax1.set_title(f'Real Satellite Image - {target_city}\nSource: {image_source}\nCoordinates: {latitude:.4f}, {longitude:.4f}', fontsize=12, weight='bold')
    ax1.axis('off')
    
    # Grayscale version for cloud analysis
    ax2 = plt.subplot(2, 3, 3)
    img_gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    ax2.imshow(img_gray, cmap='gray')
    ax2.set_title('Grayscale Analysis')
    ax2.axis('off')
    
    # Cloud density visualization
    ax3 = plt.subplot(2, 3, 4)
    ax3.imshow(img_gray, cmap='Blues')
    ax3.set_title('Cloud Density Map')
    ax3.axis('off')
    
    # Edge detection for pattern analysis
    ax4 = plt.subplot(2, 3, 5)
    edges = cv2.Canny(img_gray, 50, 150)
    ax4.imshow(edges, cmap='Reds')
    ax4.set_title('Pattern Detection')
    ax4.axis('off')
    
    # Histogram of pixel intensities
    ax5 = plt.subplot(2, 3, 6)
    ax5.hist(img_gray.flatten(), bins=50, alpha=0.7, color='skyblue')
    ax5.set_title('Pixel Intensity Distribution')
    ax5.set_xlabel('Intensity')
    ax5.set_ylabel('Frequency')
    ax5.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Detailed image analysis
    cloud_coverage = np.mean(img_gray) / 255.0
    contrast = np.std(img_gray) / 255.0
    brightness = np.mean(img_array)
    
    print(f"IMAGE ANALYSIS METRICS:")
    print(f"  Cloud Coverage: {cloud_coverage:.3f} (0=clear, 1=full clouds)")
    print(f"  Contrast: {contrast:.3f} (higher = more varied patterns)")
    print(f"  Average Brightness: {brightness:.1f}")
    print(f"  Image Dimensions: {img.size[0]} x {img.size[1]} pixels")
    print(f"  Color Mode: {img.mode}")
    print(f"  File Path: {satellite_image_path}")
    
    # Weather pattern interpretation
    print(f"\nWEATHER PATTERN INTERPRETATION:")
    if cloud_coverage > 0.7:
        print("  Heavy cloud cover detected - possible storm system")
    elif cloud_coverage > 0.4:
        print("  Moderate cloud cover - scattered clouds")
    else:
        print("  Light cloud cover - mostly clear skies")
    
    if contrast > 0.3:
        print("  High contrast patterns - possible organized weather system")
    else:
        print("  Low contrast - uniform weather conditions")

else:
    print("No satellite image available for detailed analysis")
    print("Possible reasons:")
    print("  - API key not configured in .env.local")
    print("  - Network connectivity issues")
    print("  - API service temporarily unavailable")

No satellite image available for detailed analysis
Possible reasons:
  - API key not configured in .env.local
  - Network connectivity issues
  - API service temporarily unavailable


In [46]:
print(f"Comprehensive Cyclone Prediction for {target_city}")
print("="*60)

# Combine predictions
weather_available = weather_prediction is not None
satellite_available = satellite_prediction is not None

if weather_available and satellite_available:
    combined_prediction = 0.6 * weather_prediction + 0.4 * satellite_prediction
    confidence_level = "HIGH"
    method = "Weather + Satellite Combined"
elif weather_available:
    combined_prediction = weather_prediction
    confidence_level = "MEDIUM"
    method = "Weather Only"
elif satellite_available:
    combined_prediction = satellite_prediction
    confidence_level = "MEDIUM"
    method = "Satellite Only"
else:
    combined_prediction = 0.5
    confidence_level = "LOW"
    method = "No Model Available"

final_risk_level = config.get_risk_level(combined_prediction)

print(f"Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Location: {target_city}")
print(f"Coordinates: {latitude:.4f}, {longitude:.4f}" if latitude else "N/A")

print(f"Individual Predictions:")
if weather_available:
    print(f"  Weather Model: {weather_prediction:.3f} ({weather_risk_level})")
if satellite_available:
    print(f"  Satellite Model: {satellite_prediction:.3f} ({satellite_risk_level})")

print(f"Combined Assessment:")
print(f"  Risk Level: {final_risk_level}")
print(f"  Probability: {combined_prediction:.3f}")
print(f"  Confidence: {confidence_level}")
print(f"  Method: {method}")

# Recommendations based on risk level
print(f"Recommendations:")
if combined_prediction >= 0.7:
    print("  HIGH RISK - Take immediate precautions")
    print("  - Monitor weather updates continuously")
    print("  - Prepare emergency supplies")
    print("  - Consider evacuation if advised")
elif combined_prediction >= 0.5:
    print("  MODERATE RISK - Stay alert")
    print("  - Monitor weather conditions regularly")
    print("  - Have emergency plans ready")
    print("  - Secure outdoor items")
else:
    print("  LOW RISK - Normal conditions")
    print("  - Continue regular activities")
    print("  - Maintain weather awareness")

# Save results
prediction_results = {
    'timestamp': datetime.now().isoformat(),
    'location': target_city,
    'coordinates': {'lat': latitude, 'lon': longitude} if latitude else None,
    'weather_prediction': weather_prediction,
    'satellite_prediction': satellite_prediction,
    'combined_prediction': combined_prediction,
    'risk_level': final_risk_level,
    'confidence': confidence_level,
    'method': method
}

results_filename = f"logs/cyclone_prediction_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(results_filename, 'w') as f:
    json.dump(prediction_results, f, indent=2)

print(f"Results saved to: {results_filename}")
print("Cyclone prediction analysis complete")

Comprehensive Cyclone Prediction for Visakhapatnam
Analysis Time: 2025-07-06 17:35:53
Location: Visakhapatnam
Coordinates: 17.6900, 83.2093
Individual Predictions:
  Weather Model: 0.520 (MODERATE)
Combined Assessment:
  Risk Level: MODERATE
  Probability: 0.520
  Confidence: MEDIUM
  Method: Weather Only
Recommendations:
  MODERATE RISK - Stay alert
  - Monitor weather conditions regularly
  - Have emergency plans ready
  - Secure outdoor items
Results saved to: logs/cyclone_prediction_20250706_173553.json
Cyclone prediction analysis complete
